<img src="images/mcg.jpg", style="width: 100px">

# Bayesian Networks with PGMPY

We shall use the restaurant traffic model to determine how busy a restaurant is given that a new dish has been introduced.

<img src="images/restaurant_traffic.png", style="width: 600px;">

pgmpy is an open source library for working with Probablistic Graphical Models. Install pgmpy with the command:

```python
pip install pgmpy
```

Verify pgmpy is installed:

In [1]:
!pip install pgmpy

## Create a Bayesian Network Object

### Specifying the Structure of the Model

To create a Bayesian Network Model, you need to specify Nodes (Random Variables) in the heirarchical order of influence. We know the dependency chain as:

* Festival, Game -> Traffic
* Sunny, Rain -> Pedestrian
* New Dish(Dose), Traffic, Pedestrian -> Restaurant (Vidyarthi Bhavan).

In [3]:
from pgmpy.models import BayesianModel

restaurant_model = BayesianModel([('F', 'T'),('G', 'T'), ('S', 'P'),
                                  ('R', 'P'), ('ND', 'B'), ('T', 'B'),
                                  ('P', 'B')])

We have now created nodes and directed edges of the Bayesian network.

### Specifying Conditional probabilites of each Nodes (Random Variables) 

The probability that there is an accident on the road leading to the restaurant is 0.05. Hence, we shall assign the CPDs accordingly. Simiarly, we shall assign CPDs for all variables:

Let us look at variables that can affect traffic patterns. These could be stochastic (time variant) as well, however for the current example, let us assume the september season with Krishna janmashtami and Gauri/Ganesha and assign a higher probability for a festival.

* Festival (boolean states) 

|      CPD               |    No      |    Yes   |
|------------------------|------------|----------|     
| Festival               |    0.7     |    0.3   | 


* Game (boolean state)

|      CPD               |    No      |    Yes   |
|------------------------|------------|----------|     
| Game.                  |    0.95    |     0.05 | 

* Assuming that Bengaluru would be medium sunny most of the time,

|      CPD               |    Light   |  Medium  | High  |
|------------------------|------------|----------|-------|
| Sunny                  |    0.2     |   0.7    | 0.1   |

* Assuming that in the monsoon season, we have either light or medium rain most of the time,

|      CPD               |    Light   |  Medium  | High  |
|------------------------|------------|----------|-------|
| Rain                   |    0.4     |   0.5    | 0.1   |

In [11]:
from pgmpy.factors.discrete import TabularCPD

festival_cpd = TabularCPD(variable='F',
                       variable_card=2,
                       values=[[.7, .3]])

game_cpd = TabularCPD(variable='G',
                       variable_card=2,
                       values=[[.95, .05]])

sunny_cpd = TabularCPD(variable='S',
                       variable_card=3,
                       values=[[0.2, 0.7, 0.1]])

rain_cpd = TabularCPD(variable='R',
                       variable_card=3,
                       values=[[0.4, 0.5, 0.1]])

new_dish = TabularCPD(variable='ND',
                       variable_card=2,
                       values=[[0.2, 0.8]])

traffic_cpd = TabularCPD(variable='T',
                         variable_card=3,
                         values=[[.8, .6, .3, .1 ],
                                 [.16, .25, .4, .2],
                                 [.04, .15, .3, .7]],
                         evidence=['F', 'G'],
                         evidence_card=[2, 2])

pedestrian_cpd = TabularCPD(variable='P',
                            variable_card=3,
                            values=[[.8, .6, .3, .1 ],
                                    [.16, .25, .4, .2],
                                    [.04, .15, .3, .7]],
                            evidence=['F', 'G'],
                            evidence_card=[2, 2])

# restaurant_cpd = 